# Multiprocessing Example with Stochastic Simulations

By: *Tyler Biggs*

This notebook covers my exploration into Python's multiprocessing package.

**Goals**

+ Define the stochastic process to be simulated.
+ Create a linear method and time it for reference.
+ Create a multiprocessing method and time it.

**Further Goals**
+ Upate a plot as the multiprocessing pool produces results.

#### Notebook Setup

In [1]:
# Standard Python packages.
import multiprocessing as mp
import timeit
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Import my custom packages.
from numpyGillespie import CompleteGillespie as cg
from numpyGillespie import pandas_output

### Define a Stochastic Process

#### Exponential Decay

$ A \xrightarrow{c} B $

In [3]:
# Define the process in Python.
# These are the inputs accepted by my Gillespie package.
tracked_species = [1000]

rate_constants = [0.5]

# Define the changes as full functions. Lambdas do not work with the
# multiprocessing package.
def exp_change_A(species_list):
    """Change in species induced by reaction 1."""
    return [species_list[0] - 1]

def exp_perm_A(species_list):
    """Available permutations for reaction 1."""
    return species_list[0]

### Linear Method

In [4]:
def linear_complete_gsa(runs):
    exp_runs = [cg(tracked_species, rate_constants, [exp_change_A], [exp_perm_A], max_sim_rxn=1000).simulate() for i in range(runs)]
    return exp_runs

In [5]:
%timeit -n1 -r4 linear_complete_gsa(100)

TypeError: list indices must be integers or slices, not tuple

### Multiprocessing Method

To pass this to `pool.apply_sync` it seems we need a function that returns a function.

In [6]:
def GillespieProcess(species, rates, species_changes, permutations, max_sim_rxn):
    return cg(species, rates, species_changes, permutations, max_sim_rxn).simulate()

In [7]:
def pool_build_complete_gsa(processes, runs):
    pool = mp.Pool(processes=processes)
    results = [pool.apply_async(GillespieProcess, args=(tracked_species, rate_constants, [exp_change_A], [exp_perm_A], 1000)) for i in range(runs)]
    results = [p.get() for p in results]
    return results

In [8]:
%timeit -n1 -r4 pool_build_complete_gsa(3, 100)

TypeError: list indices must be integers or slices, not tuple